In [2]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import cv2
from glob import glob
from natsort import natsorted
import time
import os
import shutil


from sklearn.utils import shuffle

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras import losses

import sys
sys.path.append('/home/VirtualFlaw/Data')
import preprocessing.pipe as pipe
sys.path.append('/home/VirtualFlaw/Data/refactoring/src/Model')


2023-03-14 15:21:24.205047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 15:21:24.331338: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """

    noise_factor = 0.4
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)

In [4]:
#THICKNESS_LIST = "ALL"
THICKNESS_LIST = [7.92, 8.74, 9.53]
CATEGORY_LIST = pipe.manage_category_thickness(thickness_range=THICKNESS_LIST)

image_path_list = []
for category in CATEGORY_LIST:
    thikness =  pipe.manage_category_thickness(category=category)
    image_list = glob(f'/home/VirtualFlaw/Data/IMAGE/Raw_jpg_1024/{thikness}/{category}/*.jpg')
    image_list = natsorted(image_list)
    image_path_list += image_list[3:-3] #Exclude 3 images - EndTap, IQI

if len(image_path_list) > 50000:
    image_path_list = np.random.choice(image_path_list, 50000, replace=False)
    
image_path_list = shuffle(image_path_list)
for image in image_path_list:
    image_name = image.split('/')[-1]
    if not image_name[-5].isdigit():
        image_path_list.remove(image)
        
    
print(f"Total images: {len(image_path_list)}")
    
            
                

Total images: 0


In [4]:
IMG_WIDTH = 1024
IMG_HEIGHT = 1024
IMG_CHANNELS = 1


MAX_IMAGE =  2000

today = time.strftime('%Y%m%d %H%M', time.localtime(time.time()))
os.makedirs(f'/home/VirtualFlaw/Data/refactoring/logs/denoising/{today}', exist_ok=True)
shutil.copy('/home/VirtualFlaw/Data/refactoring/src/ImageDenoising.ipynb', f'/home/VirtualFlaw/Data/refactoring/logs/denoising/{today}/ImageDenoising.ipynb')


X_train = np.zeros((MAX_IMAGE, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((MAX_IMAGE, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

In [5]:
for n, path in tqdm(enumerate(image_path_list), total=MAX_IMAGE):
    try:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        #image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        image = np.expand_dims(image, axis=-1)
        X_train[n] = image
        Y_train[n] = image
    except:
        break

100%|██████████| 2000/2000 [00:09<00:00, 209.18it/s]


In [6]:
X_train = noise(X_train)

In [7]:

print("X_train.shape : ", X_train.shape)
print("Y_train.shape : ", Y_train.shape)


X_train.shape :  (2000, 1024, 1024, 1)
Y_train.shape :  (2000, 1024, 1024, 1)


In [8]:

learning_rate = 1e-5
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    #sequential model
    model = tf.keras.Sequential()
    #normalization
    model.add(layers.BatchNormalization(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same',))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same',))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
    
    earlystopper = EarlyStopping(patience=10, verbose=1)
    checkpointer = ModelCheckpoint(f'/home/VirtualFlaw/Data/refactoring/logs/denoising/{today}/model.h5', verbose=1, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1)
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=losses.MeanSquaredError(), metrics=['accuracy'])
    
    
    

2023-03-11 19:17:22.867495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 19:17:24.462504: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22294 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:18:00.0, compute capability: 8.6
2023-03-11 19:17:24.463604: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22294 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6
2023-03-11 19:17:24.464590: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localho

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localho

In [9]:

result = model.fit(X_train, Y_train, validation_split=0.3, batch_size=3, epochs=300, callbacks=[earlystopper, checkpointer, reduce_lr])



Epoch 1/300
INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1


2023-03-11 19:17:48.731005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-11 19:17:50.077546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-11 19:17:51.677372: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-11 19:17:57.525922: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.52GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-11 19:17:57.525962: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.52GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-11 19:18:00.710609: I tensorflow

467/467 [==============================] - ETA: 0s - loss: 8805.4277 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 8775.78516, saving model to /home/VirtualFlaw/Data/refactoring/logs/denoising/20230311 1916/model.h5
467/467 [==============================] - 162s 300ms/step - loss: 8805.4277 - accuracy: 0.0000e+00 - val_loss: 8775.7852 - val_accuracy: 0.0000e+00 - lr: 1.0000e-05
Epoch 2/300


2023-03-11 19:20:22.580548: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.01GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  1/467 [..............................] - ETA: 2:21 - loss: 8478.5820 - accuracy: 0.0000e+00

...

  6/467 [..............................] - ETA: 2:20 - loss: 8339.4932 - accuracy: 0.0000e+00

2023-03-11 19:20:24.407093: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.01GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  7/467 [..............................] - ETA: 2:19 - loss: 8368.1816 - accuracy: 0.0000e+00

2023-03-11 19:20:24.709381: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.01GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


467/467 [==============================] - ETA: 0s - loss: 8805.2178 - accuracy: 0.0000e+00
Epoch 2: val_loss improved from 8775.78516 to 8775.57031, saving model to /home/VirtualFlaw/Data/refactoring/logs/denoising/20230311 1916/model.h5
467/467 [==============================] - 159s 340ms/step - loss: 8805.2178 - accuracy: 0.0000e+00 - val_loss: 8775.5703 - val_accuracy: 0.0000e+00 - lr: 1.0000e-05
Epoch 3/300
467/467 [==============================] - ETA: 0s - loss: 8804.9990 - accuracy: 0.0000e+00
Epoch 3: val_loss improved from 8775.57031 to 8775.35352, saving model to /home/VirtualFlaw/Data/refactoring/logs/denoising/20230311 1916/model.h5
467/467 [==============================] - 159s 341ms/step - loss: 8804.9990 - accuracy: 0.0000e+00 - val_loss: 8775.3535 - val_accuracy: 0.0000e+00 - lr: 1.0000e-05
Epoch 4/300
467/467 [==============================] - ETA: 0s - loss: 8804.7783 - accuracy: 0.0000e+00
Epoch 4: val_loss improved from 8775.35352 to 8775.13770, saving model to 

In [ ]:
model = tf.keras.models.load_model(f"/home/VirtualFlaw/Data/refactoring/logs/denoising/{today}/model.h5")
image_path = "/home/VirtualFlaw/Data/Raw_jpg/202212/REJ/7.92/22I002-02-012/22I002-02-012-1st-1-(1)-38-R-UC.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
image = np.expand_dims(image, axis=-1)
image = np.expand_dims(image, axis=0)
image = image.astype(np.float32)
image = image / 255.0

pred = model.predict(image)
pred = cv2.normalize(pred, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

cv2.imwrite(f"/home/VirtualFlaw/Data/refactoring/logs/denoising/{today}/denoised.jpg", pred[0])

OSError: No file or directory found at /home/VirtualFlaw/Data/refactoring/logs/denoising/20230311 1859/model.h5